## Downloading dependencies and code

## Mounting drive, extracting dataset and weights

## Training PlaneRCNN

In [1]:
from fox.config import Config
from fox.dataset import ComboDataset
from fox.model import Model
import torch
import pytorch_lightning as pl
from athena.utils.progbar import ProgbarCallback
from fox.utils import parse_data_cfg
from fox.yolov3.utils.datasets import LoadImagesAndLabels

In [2]:
config = Config(
    USE_YOLO=True,
    DATA_DIR="data",
    IMG_SIZE=640,
    MIN_IMG_SIZE=480,
    BATCH_SIZE=1,
    MIDAS_LOSS_WEIGHT=1,
    PLANERCNN_LOSS_WEIGHT=1,
    PREDICT_DEPTH=True,
    EPOCHS=1
)
dataset = ComboDataset(config)
val_dataset = ComboDataset(config, train=False)

loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=True,
    collate_fn=dataset.collate_fn
)
loader_val = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=config.BATCH_SIZE,
    collate_fn=val_dataset.collate_fn
)

Caching labels data\yolo\labels.npy (1001 found, 0 missing, 14 empty, 0 duplicate, for 1015 images): 100%|██████████| 1015/1015 [00:00<00:00, 18796.25it/s]
Caching labels data\yolo\labels.npy (99 found, 0 missing, 0 empty, 0 duplicate, for 99 images): 100%|██████████| 99/99 [00:00<00:00, 3666.58it/s]


In [3]:
model = Model(config, len(loader), 4, dataset.yolo_dataset.yolo_labels)

Using cache found in C:\Users\guddu/.cache\torch\hub\facebookresearch_WSL-Images_master
Loading weights:  weights/midas.pt
Model Summary: 147 layers, 5.93877e+07 parameters, 5.93877e+07 gradients
Using cache found in C:\Users\guddu/.cache\torch\hub\ultralytics_yolov3_master

                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     20672  models.common.Bottleneck                [64, 64]                      
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    164608  models.common.Bottleneck                [128, 128]                    
  5                -1  1    295424  models.common.Conv                      [128, 

In [4]:
pl.seed_everything(0)
trainer = pl.Trainer(
    max_epochs=config.EPOCHS,
    gpus=1,
    callbacks=[ProgbarCallback()],
    progress_bar_refresh_rate=20,
    automatic_optimization=False
)

trainer.fit(
    model,
    train_dataloader=loader,
    val_dataloaders=loader_val
)

Global seed set to 0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
C:\ProgramData\Anaconda3\lib\site-packages\pytorch_lightning\utilities\distributed.py:68: UserWarning: Disable automatic optimization with the trainer flag is deprecated and will be removed in v1.3.0!Please use the property on the LightningModule for disabling automatic optimization
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type          | Params
---------------------------------------------------------
0 | midas_net              | MidasNet      | 105 M 
1 | yolo_part              | YoloPart      | 64.5 M
2 | planercnn_model        | MaskRCNN      | 24.3 M
3 | planercnn_refine_model | RefineModel   | 979 K 
4 | planercnn_head         | PlaneRCNNHead | 78.0 M
---------------------------------------------------------
167 M     Trainable params
105 M     Non-trainable params
273 M     Total params
1,092.801 Total estimated model param

1

In [ ]:
torch.save(model.state_dict(), "model.pt")